# CNNs for Image Classification

In this notebook I started from the model seen in class and explored the hyperparameters space manually to find the direction in which the model could give better results.

In the end I found that the only approach that resulted in a better performance was increasing the number of filters of the convolutional layers (only that model is uncommented).

The model building and training are encapsulated in functions.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
import os
import json
from datetime import datetime
from PIL import Image

import tensorflow as tf
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.constraints import MaxNorm

SEED = 1234
tf.random.set_seed(1234)

cwd = os.getcwd()

## Datasets loading

In this section all the datasets are loaded and the "dataset_split.json" file is written.

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

bs = 8
img_w = 256
img_h = 256
validation_split = 0.1

classes = [
    'owl',              # 0
    'galaxy',           # 1
    'lightning',        # 2
    'wine-bottle',      # 3
    't-shirt',          # 4
    'waterfall',        # 5
    'sword',            # 6
    'school-bus',       # 7
    'calculator',       # 8
    'sheet-music',      # 9
    'airplanes',        # 10
    'lightbulb',        # 11
    'skyscraper',       # 12
    'mountain-bike',    # 13
    'fireworks',        # 14
    'computer-monitor', # 15
    'bear',             # 16
    'grand-piano',      # 17
    'kangaroo',         # 18
    'laptop'            # 19
]

# LOAD TRAINING AND VALIDATION SETS

data_gen = ImageDataGenerator(rotation_range = 10,
                              width_shift_range = 10,
                              height_shift_range = 10,
                              zoom_range = 0.3,
                              horizontal_flip = True,
                              vertical_flip = True,
                              rescale = 1./255, 
                              validation_split = validation_split)

training_dir = os.path.join(cwd, "Classification_Dataset", "training")

print("Training")

training_generator = data_gen.flow_from_directory(training_dir,
                                        batch_size = bs,
                                        classes = classes,
                                        class_mode = 'categorical',
                                        shuffle = True,
                                        seed = SEED,
                                        subset = 'training')

print("\nValidation")

validation_generator = data_gen.flow_from_directory(training_dir,
                                        batch_size = bs,
                                        classes = classes,
                                        class_mode = 'categorical',
                                        shuffle = True,
                                        seed = SEED,
                                        subset = 'validation')

training_dataset = tf.data.Dataset.from_generator(lambda: training_generator,
                                              output_types = (tf.float32, tf.float32),
                                              output_shapes = ([None, img_w, img_h, 3], [None, len(classes)]))
validation_dataset = tf.data.Dataset.from_generator(lambda: validation_generator,
                                              output_types = (tf.float32, tf.float32),
                                              output_shapes = ([None, img_w, img_h, 3], [None, len(classes)]))

training_dataset = training_dataset.repeat()
validation_dataset = validation_dataset.repeat()

# WRITE FILENAMES TO JSON FILE

filenames = {
    "training" : {},
    "validation" : {}
}

for c in classes:
    filenames["training"][c] = []
    filenames["validation"][c] = []
    
    for fn in training_generator.filenames:
        if c in fn:
            filenames["training"][c].append(fn.replace(c + "/", ""))
    
    for fn in validation_generator.filenames:
        if c in fn:
            filenames["validation"][c].append(fn.replace(c + "/", ""))

with open('dataset_split.json', 'w') as file:
    json.dump(filenames, file, indent=4)
    

# LOAD TEST SET filenames

print("\nTo predict")

test_dir = os.path.join(cwd, "Classification_Dataset", "test")
test_filenames = next(os.walk(test_dir))[2]
test_filenames = list(filter(lambda fn: fn[-4:] == '.jpg', test_filenames))

print("Found " + str(len(test_filenames)) + " images.")

Training
Found 1406 images belonging to 20 classes.

Validation
Found 148 images belonging to 20 classes.

To predict
Found 500 images.


## Model building, fitting and predicting functions

This functions are used to build and train models and to predict the results.

In [9]:
def build_model(input_shape, 
                conv_depth, start_num_filters, kernel_size, pool_size, 
                fc_units, num_classes, kernel_regularizer = None, dropout = None):
    """
    This function is used to build automatically the model given some hyperparameters.
    
    Parameters
    ----------
    input_shape: tuple of ints
        the input_shape to be fed to the first layer of the model.
    conv_depth: int
        the number of convolutional blocks used (convolution + activation + pooling).
    start_num_filters: int
        the number of convolutional filters used in the first convolutional layer, it's multiplied by two in every subsequent layer.
    kernel_size: pair of ints
        the size of the kernels of the convolutional layers.
    pool_size: pair of ints
        the pool size for the pooling layers.
    fc_units: list of ints
        number of neurons in each hidden layer, the number of hidden layers is given by the length of this list.
    num_classes: int
        number of outputs.
    kernel_regularizer: regularizer, optional
        regularizer used in the fully connected layers.
    dropout: float, optional
        dropout ratio used in the fully connected layers.
        
    Returns
    -------
    Keras model
    """
    
    model = Sequential()
    
    kernel_constraint = MaxNorm(3) if dropout else None
    
    for i in range(conv_depth):
        if i == 0:
            model.add(Conv2D(filters = start_num_filters,
                             kernel_size = kernel_size,
                             strides = (1, 1),
                             padding = 'same',
                             activation = 'relu',
                             input_shape = input_shape))
        else:
            model.add(Conv2D(filters = start_num_filters,
                             kernel_size = kernel_size,
                             strides = (1, 1),
                             padding = 'same',
                             activation = 'relu'))
            
        model.add(MaxPool2D(pool_size = pool_size))
        start_num_filters *= 2
        
    model.add(Flatten())
    
    if dropout:
        model.add(Dropout(dropout))
    
    for i in range(len(fc_units)):
        model.add(Dense(units = fc_units[i], 
                        activation = 'relu', 
                        kernel_regularizer = kernel_regularizer,
                        kernel_constraint = kernel_constraint))
        if dropout:
            model.add(Dropout(dropout))
    
    model.add(Dense(units = num_classes, 
                    activation = 'softmax', 
                    kernel_regularizer = kernel_regularizer,
                    kernel_constraint = kernel_constraint))
    
    return model

In [10]:
def create_csv(results, results_dir='./'):
    """
    Function used to write a prediction dictionary to a csv file
    
    Parameters
    ----------
    results: dict
        predictions
    results_dir: string, optional
        the directory
    """

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [11]:
def fit_model(model, model_name = datetime.now().strftime('%b%d_%H-%M-%S')):
    """
    Function used to fit the model (and save the checkpoints).
    It saves all the checkpoints that increased the performance and returns the best one.
    The performance evaluated is the validation accuracy.
    Early stopping is used with 20 epochs of patience.
    It also uses a tensorboard callback for visualization.
    
    Parameters
    ----------
    model: keras model
        model to fit.
    model_name: string, optional
        name of the model.
    
    Returns
    -------
    keras model: the best model.
    string: the directory of the model.
    """
    
    cwd = os.getcwd()
    
    # General experiments folder
    exps_dir = os.path.join(cwd, 'classification_experiments')
    if not os.path.exists(exps_dir):
        os.makedirs(exps_dir)
    
    now = datetime.now().strftime('%b%d_%H-%M-%S')
    
    # This experiment folder
    exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
    if not os.path.exists(exp_dir):
        os.makedirs(exp_dir)
    
    # Checpoints folder
    ckpt_dir = os.path.join(exp_dir, 'ckpts')
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    # Checkpoints callback, best one will be the last saved
    ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                       save_weights_only=True,
                                                       save_best_only=True, 
                                                       monitor='val_accuracy', 
                                                       mode = 'max')
    
    # Tensorboard folder
    tb_dir = os.path.join(exp_dir, 'tb_logs')
    if not os.path.exists(tb_dir):
        os.makedirs(tb_dir)
    
    # Tensorboard callback
    tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                                 profile_batch=0,
                                                 histogram_freq=1)  # if 1 shows weights histograms
    
    # Early stopping callback
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                                   patience=20,
                                                   mode = 'max')
    
    callbacks= [ckpt_callback, tb_callback, es_callback]
    
    model.fit(x=training_dataset,
              epochs=150,
              steps_per_epoch=len(training_generator),
              validation_data=validation_dataset,
              validation_steps=len(validation_generator), 
              callbacks=callbacks)
    
    # Load best model (last one saved)
    latest = tf.train.latest_checkpoint(ckpt_dir)
    print("Latest model: " + latest)
    model.load_weights(os.path.join(ckpt_dir, latest))
    
    return (model, exp_dir)

In [12]:
def predict(model, exp_dir):
    """
    Function used to make predictions and write them to file.
    
    Parameters
    ----------
    model: keras model
    exp_dir: the directory where the predictions must be saved.
    """
    
    results = {}

    for image_name in test_filenames:
        img = Image.open(os.path.join(test_dir,image_name)).convert('RGB').resize((img_w, img_h))
        img_array = np.array(img)
        img_array = np.expand_dims(img_array, 0)
        img_array = np.divide(img_array,255)
        tensor = tf.convert_to_tensor(img_array, dtype = tf.float32)
        
        prediction = np.argmax(model.predict(tensor))
        results[image_name] = prediction

    create_csv(results = results, results_dir=exp_dir)

# Models

CNN_1: as seen in class

In [8]:
#model_name = "CNN_1"
#conv_depth = 5
#start_num_filters = 8
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        1

CNN_2: as seen in class + regularization

In [9]:
#model_name = "CNN_2"
#conv_depth = 5
#start_num_filters = 8
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#alpha = 0.01
#kernel_regularizer = tf.keras.regularizers.l2(alpha)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units, 
#                   kernel_regularizer = kernel_regularizer,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 64)       

CNN_3: 7 layers, start_num_filters = 4 [REMOVED]

CNN_4: 7 layers, start_num_filters = 4, regularization [REMOVED]

CNN_5: 6 layers, start_num_filters = 8

In [10]:
#model_name = "CNN_5"
#conv_depth = 6
#start_num_filters = 8
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 64)       

CNN_6: 6 layers, start_num_filters = 8, regularization

In [11]:
#model_name = "CNN_6"
#conv_depth = 6
#start_num_filters = 8
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#alpha = 0.01
#kernel_regularizer = tf.keras.regularizers.l2(alpha)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units, 
#                   kernel_regularizer = kernel_regularizer,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 32, 64)       

CNN_7: as seen in class, start_num_filters = 16

In [12]:
model_name = "CNN_7"
conv_depth = 5
start_num_filters = 16
kernel_size = (3, 3)
pool_size = (2, 2)
fc_units = [512]
num_classes = len(classes)

lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
loss = tf.keras.losses.CategoricalCrossentropy()
metrics = ['accuracy']

model = build_model(input_shape = (img_w, img_h, 3),
                   conv_depth = conv_depth,
                   start_num_filters = start_num_filters,
                   kernel_size = kernel_size,
                   pool_size = pool_size,
                   fc_units = fc_units,
                   num_classes = num_classes)

model.compile(optimizer = optimizer,
             loss = loss,
             metrics = metrics)

model.summary()

(model, exp_dir) = fit_model(model = model, model_name = model_name)

predict(model = model, exp_dir = exp_dir)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 32, 32, 128)      

CNN_8: as seen in class, start_num_filters = 16, regularization

In [13]:
#model_name = "CNN_8"
#conv_depth = 5
#start_num_filters = 8
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#alpha = 0.01
#kernel_regularizer = tf.keras.regularizers.l2(alpha)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units, 
#                   kernel_regularizer = kernel_regularizer,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d_27 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_28 (Conv2D)           (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_29 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_29 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 32, 32, 64)       

CNN_9: as seen in class, FC has 2 hidden layers of 256 units

In [14]:
#model_name = "CNN_9"
#conv_depth = 5
#start_num_filters = 8
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [256, 256]
#num_classes = len(classes)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 32, 32, 64)       

CNN_10: as seen in class, FC has 2 hidden layers of 256 units + regularization

In [15]:
#model_name = "CNN_10"
#conv_depth = 5
#start_num_filters = 8
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [256, 256]
#num_classes = len(classes)
#
#alpha = 0.01
#kernel_regularizer = tf.keras.regularizers.l2(alpha)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units, 
#                   kernel_regularizer = kernel_regularizer,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_37 (Conv2D)           (None, 256, 256, 8)       224       
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 128, 128, 16)      1168      
_________________________________________________________________
max_pooling2d_38 (MaxPooling (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_39 (Conv2D)           (None, 64, 64, 32)        4640      
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 32, 32, 64)       

CNN_11: start_num_filters = 16, fc_units = [256,256]

In [8]:
#model_name = "CNN_11"
#conv_depth = 5
#start_num_filters = 16
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [256, 256]
#num_classes = len(classes)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       7

CNN_12: start_num_filters = 16, fc_units = [256,256], regularization [REMOVED]

CNN_13: start_num_filters = 16, conv_depth = 6

In [10]:
#model_name = "CNN_13"
#conv_depth = 6
#start_num_filters = 16
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 128)      

CNN_14: start_num_filters = 16, conv_depth = 6, regularization

In [11]:
#model_name = "CNN_14"
#conv_depth = 6
#start_num_filters = 16
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#alpha = 0.01
#kernel_regularizer = tf.keras.regularizers.l2(alpha)
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes,
#                   kernel_regularizer = kernel_regularizer)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 32, 32, 128)      

CNN_15: start_num_filters = 16, kernel_size = (5,5) [REMOVED]

CNN_16: start_num_filters = 16, dropout = 0.3

In [13]:
#model_name = "CNN_16"
#conv_depth = 5
#start_num_filters = 16
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#dropout = 0.3
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes,
#                   dropout = dropout)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 128)       7

CNN_17: start_num_filters = 16, fc_units = [512, 512], dropout = 0.3

In [14]:
#model_name = "CNN_17"
#conv_depth = 5
#start_num_filters = 16
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512, 512]
#num_classes = len(classes)
#
#dropout = 0.3
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes,
#                   dropout = dropout)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 32, 32, 128)      

CNN_18: start_num_filters = 16, dropout = 0.3, regularization

In [10]:
#model_name = "CNN_18"
#conv_depth = 5
#start_num_filters = 16
#kernel_size = (3, 3)
#pool_size = (2, 2)
#fc_units = [512]
#num_classes = len(classes)
#
#dropout = 0.3
#alpha = 0.01
#kernel_regularizer = tf.keras.regularizers.l2(alpha)
#
#lr = 1e-3
#optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
#loss = tf.keras.losses.CategoricalCrossentropy()
#metrics = ['accuracy']
#
#model = build_model(input_shape = (img_w, img_h, 3),
#                   conv_depth = conv_depth,
#                   start_num_filters = start_num_filters,
#                   kernel_size = kernel_size,
#                   pool_size = pool_size,
#                   fc_units = fc_units,
#                   num_classes = num_classes,
#                   kernel_regularizer = kernel_regularizer,
#                   dropout = dropout)
#
#model.compile(optimizer = optimizer,
#             loss = loss,
#             metrics = metrics)
#
#model.summary()
#
#(model, exp_dir) = fit_model(model = model, model_name = model_name)
#
#predict(model = model, exp_dir = exp_dir)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 256, 256, 16)      448       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 128, 128, 16)      0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 128, 128, 32)      4640      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 32, 32, 128)      